In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from openai import OpenAI, RateLimitError
import random
import time
from typing import List

class OpenAiService:
    def __init__(self):
        # 環境変数OPENAI_API_KEYが必要
        self.client = OpenAI()
        self.poll_interval_seconds: float = 10.0
        # embeddings batch の制限(最大 50,000 inputs)に合わせる用途
        self.max_inputs_per_batch: int = 50_000


    def get_chat(self, messages: list[dict], model: str = "gpt-5-nano"):
        response = self.client.chat.completions.create(
            messages=messages,
            model=model,
            temperature=1,
        )
        return response.choices[0].message.content

    def get_embeddings(
        self,
        inputs: List[str],
        *,
        model: str = "text-embedding-3-small",
        max_retries: int = 3,
        base_sleep: float = 0.5,
    ) -> list[list[float]]:
        """
        Embeddings API をリトライ付きで呼ぶ。
        - 429 / 5xx / 一時的ネットワークエラーを想定して指数バックオフ。
        - 永続的エラー（不正な入力など）は即失敗させる。
        """
        last_err: Exception | None = None
        if self.max_inputs_per_batch < len(inputs):
            raise ValueError("inputs size error.")

        for attempt in range(max_retries + 1):
            try:
                resp = self.client.embeddings.create(model=model, input=inputs)
                # 念のため data の順序を入力順に整列（多くの場合そのままだが保険）
                data_sorted = sorted(resp.data, key=lambda d: d.index)
                return [d.embedding for d in data_sorted]
            except RateLimitError as e:
                # 制限エラーは即時
                raise e
            except Exception as e:
                last_err = e

                # ここでは例外型を細かく分岐しない（SDKの例外体系が変わる可能性があるため）
                # 代わりに「最後の数回は諦める」「待って再試行」という運用上安全な方針にする。
                if attempt >= max_retries:
                    raise last_err

                # 指数バックオフ + ジッター
                sleep = base_sleep * (2**attempt)
                sleep *= 1.0 + random.random() * 0.2  # 0〜20%ジッター
                time.sleep(sleep)

In [12]:
service = OpenAiService()
res = service.get_chat([
    {"role": "system", "content": "貴方は国会の発言をまとめています。次のような発言を議員がおこないました。議員の質疑の趣旨を600文字以内に簡潔にまとめなさい。"},
    {"role": "user", "content": "お招きがあったら行ってくださいね。それは総理が行ってくれたら全然違うと思いますね。この前、福島第一原発の廃炉処理の皆さんに激励いただいたことは、本当に現場も勇気が出たと思います。一国のトップがそういうメッセージを発することは非常に大切だと思いますし、繰り返しになりますけれども、原子力政策は国策としてやってきたんですよ。やはり国がしっかり責任を持つことを総理には御理解いただいていると思うので、柏崎刈羽、いつか是非行っていただきたいと改めて要請しておきたいと思います。 　次に、賃金を上げることもやはり大事だと思うんですね。本当に、働いている人が元気になっていくことが必要なんです。 　この賃上げなんですけれども、この間、日本企業、利益はかなり出すようになっているんですが、配当とそして自社株買いに物すごくお金を使っています。ＲＯＥは自社株買いをすると計算上は上がりますけれども、そのお金をもっと従業員の給料とか、例えば、さっきは国の投資の話をしましたけれども、民間もやはり投資に回すべきだと思うんですね。 　商法の時代は、実は、自社株買いは商法上禁止されていたんですけれども、これがだんだん緩和されて、そのことで日本の株式市場も上がって、外国の方には評価をいただいているんですが、多分、三十兆円ぐらいになると思うんです、今年、自社株買いに回すお金は。せめて、そのうち、例えば十兆、十兆、十兆ぐらいで、十兆円分をもっと給与のアップとか、十兆円分をもっと投資の方に回せば、本当の意味での経済の成長につながっていくと思いますが。 　禁止しろとは言いませんけれども、何か東京証券取引所にも協力いただく必要があると思うんですが、会社はもちろん株主のものだという考え方もあるんですけれども、会社はみんなのものですよ。働いている従業員、取引先。そんな、みんながハッピーになるような果実の配分のやり方に変えていくことが私は中長期的な日本の成長につながると思いますが、いかがですか？　やはりマーケットにも、給料を増やすとか、賃上げをする、人材投資をすることが株価に長期でプラスに響いてくるんだということも、開示情報などにきちんと明記するとか、市場がプラスに評価するような資源配分の在り方を国も、また証券取引所も促していくということは私は必要だと思うので、是非、今おっしゃったような方向での改革を後押しをいただきたいなと思います。 　もう一つ、不動産について。 　この前、ネットの記事を見ていたら、所得の十倍以上払わないと新築マンションを買えない、東京だけじゃなくて二十四都道府県でそうなっていると。一番低かったのは私の香川県だったんですが。東京だと所得の十七倍じゃないと新築マンションを買えないというのは、ちょっと、中間層の人からすると、ちゃんとした家を持つことが物すごく夢物語になっちゃっていて、これは私はどこかで変えなきゃいけないと思っているんですね。 　いろいろな理由で上がっていますけれども、一つが不動産プリセールといって、引渡し前に、その引き渡す権利自体を売買する。これは中国で盛んにやられていたのが禁止されたので、そういった投資マネーが、今、日本に流れてきているんじゃないか。課税当局もなかなか、この売買で得た所得からちゃんと課税するのは難しいと思います。 　ですから、業界の自主規制で一定程度規制を入れてきているんですけれども、国も、引渡し前の売買、これは一定の規制を入れるべきではないでしょうか。あるいは、バブル期にあったように、二年未満の短期の、明らかに住むためじゃなくて投機目的のやり取りについては通常の譲渡益課税よりも少し重課して税金をいただくような、こういう仕組みで、少し過熱している不動産価格の上昇を抑えるような対策が必要だと思いますが、いかがでしょうか？　これは、まず実態把握をしっかりやってもらいたい。やはり、それで売買して利益を得ても、まず実態把握しないと課税できないですよ。真面目に日本人だけ払って、非居住者だったら逃げてしまって追えないみたいになると、非常に不公平にもなりますからね。そこは是非進めていただきたい。 　最後に、台湾有事の国会答弁でいろいろ問題になっていますが、撤回する必要はありません。毅然と日本の立場をこれからも説明していってほしいと思います。 　ただ、今、中国が情報戦をやっています。ですから、総理にお願いしたいのは、改めて日米間の連携を一層強化してもらいたい。四月にトランプ大統領が訪中しますが、その前に、何らかの形でもう一度日米首脳会談をやっていただきたいなと思います。 　昨日、ダボス会議の、ワールド・エコノミック・フォーラムのボルゲさんが来て、話をしたら、トランプさんは五人の閣僚を連れて、一月、ダボス会議に行くそうです。中国も首脳が行くそうなので、国会の日程がいつも問題になるんですが、我々は協力しますから、総理、ダボス会議に行って、その場でもいいので、総理と、片山大臣も多分行かれると思いますけれども、もう一回、日米の揺るぎないきずなをそこで確認する、これを是非やるべきだと思いますが、いかがですか。"},
])
print(res)

- 原子力政策は国策として推進してきたとの認識を reaffirm し、総理の発信力を活かして福島第一の廃炉現場を激励した意義を強調。柏崎刈羽の訪問を改めて要請。

- 賃上げと人材投資の重要性を指摘。自社株買い・配当に偏る資金を給与・投資へ回す改革を、市場開示と証券取引所の協力で促進する必要性を提言。

- 不動産価格の過熱抑制を主張。所得対比の住宅購入難、引渡し前取引の規制強化、短期投機には重課税等の措置を検討・実態把握を徹底するべきと提案。

- 台湾有事への対応と日米連携の強化を要請。日米首脳会談の実現とダボス会議での日米関係の揺るぎない結束の再確認を求める。


In [103]:
class KokkaiOpenAIService(OpenAiService):
    def get_issue_summary(self, issue_speeches: List[str]) -> str:
        speech = "\n".join(issue_speeches)[:80000]  # 8万文字程度で制限
        messages = [
            {
                "role": "system",
                "content": (
                    "貴方は国会の発言をまとめています。次のような発言を議員がおこないました。"
                    "議員の質疑の趣旨を600文字以内に簡潔にまとめなさい。"
                ),
            },
            {"role": "user", "content": speech}
        ]
        return self.get_chat(messages)

    def get_speaker_summary(self, issue_summaries: List[str]) -> str:
        content = "\n".join(issue_summaries)[:80000]  # 8万文字程度で制限
        messages = [
            {
                "role": "system",
                "content": (
                    "貴方は選挙用に国会議員の政策や考えをまとめています。当該議員は国会で次のような質疑をおこなっています。"
                    "有権者にわかりやすいようにどのような政策を重視しているかを整理してビラ用に400文字以内でまとめてください。"
                ),
            },
            {"role": "user", "content": content}
        ]
        return self.get_chat(messages)

In [3]:
import requests
from typing import Optional, Iterator
import time
from pydantic import BaseModel

class SpeakerItem(BaseModel):
    speaker: Optional[str] = None
    speakerYomi: Optional[str] = None
    speechID: str
    issueID: str
    imageKind: str
    searchObject: int
    session: int
    nameOfHouse: Optional[str] = None
    nameOfMeeting: Optional[str] = None
    issue: Optional[str] = None
    date: Optional[str] = None
    speechOrder: int
    speakerGroup: Optional[str] = None
    speakerPosition: Optional[str] = None
    speakerRole: Optional[str] = None
    speech: str
    startPage: int = 0
    speechURL: Optional[str] = None
    meetingURL: Optional[str] = None
    pdfURL: Optional[str] = None



# 参考
# https://kokkai.ndl.go.jp/api.html
class KokkaiSpeacherApi:
    def __init__(self):
        self.endpoint = "https://kokkai.ndl.go.jp/api/speech"
        self.next_request_time = time.time()
        self.interval_sec = 2
        self.max_records = 100

    def get_speak(self, speaker:str, from_date: Optional[str]=None, until_date: Optional[str]=None)->Iterator[SpeakerItem]:
        with requests.Session() as session:
            # 必要なら User-Agent を明示（礼儀）
            session.headers.update({
                "User-Agent": "kokkai-api-client/1.0"
            })
            params = {
                "speaker": speaker,
                "maximumRecords": self.max_records,
                "recordPacking": "json",
                "startRecord": 1
            }
            if from_date:
                params["from"] = from_date
            if until_date:
                params["until"] = until_date
            yield from self._call_api(session, params)
    
    def _call_api(self, session: requests.Session, params: dict)->Iterator[SpeakerItem]:
        current_time = time.time()
        if current_time < self.next_request_time:
            time.sleep(self.next_request_time-current_time)
        response = session.get(self.endpoint, params=params)
        self.next_request_time = time.time() + self.interval_sec
        response.raise_for_status()
        result = response.json()
        for record in result.get("speechRecord", []):
            item = SpeakerItem(**record)
            yield item
        if result.get("nextRecordPosition"):
            params["startRecord"] = result["nextRecordPosition"]
            yield from self._call_api(session, params)


In [78]:
from_date = "2022-01-01"
until_date = "2026-01-31"
ctrl = KokkaiSpeacherApi()
for rec in ctrl.get_speak("うるま譲司", from_date=from_date, until_date=until_date):
    print(rec)

speaker='うるま譲司' speakerYomi='うるまじょうじ' speechID='121904889X00720251211_077' issueID='121904889X00720251211' imageKind='会議録' searchObject=77 session=219 nameOfHouse='衆議院' nameOfMeeting='内閣委員会' issue='第7号' date='2025-12-11' speechOrder=77 speakerGroup='日本維新の会' speakerPosition=None speakerRole=None speech='○うるま委員\u3000日本維新の会のうるま譲司です。\r\n\u3000今回、公務員の給与法の法案は、優秀な人材確保と構造的な賃上げを実現する上で重要なものであると理解しております。しかし、公務員の給与は、主に国民の皆様からお預かりした税金で支えられております。国民の納得を得られているのかどうか、理解を得るためにしっかりと、我々も、そして政府も含めて努力できているのかどうか、そういったところを中心にお伺いさせていただきます。\r\n\u3000まずは、人事院にお伺いいたします。\r\n\u3000今回、人事院が、比較対象企業規模を五十人以上から百人以上に引き上げるという比較方法の見直しを行いました。しかし、全労働者の七割は中小企業で働いております。また、国民の皆さんは、公務員といえば倒産しない、首にならない、そういった見方をされている方も多くおられます。そういった観点も入れて、そういった中で、この比較方法の見直しに関して、国民の納得感、理解を得るための努力をしっかりやっているのか、どのような方策をやっているのか、まずはお伺いいたします。\r\n\u3000そして、あわせて、今回の比較方法の見直しは、行政課題の複雑化や激しい人材獲得競争を踏まえた判断とのことでありますが、この見直しが実際に人材獲得競争に与える効果について、今後どのように分析、検証していくのか、見解をお伺いいたします。' startPage=0 speechURL='https://kokkai.ndl.go.jp/txt/121904889X00720251211/77' meetingURL='htt

In [126]:
import duckdb


class SpeakerDbItem(SpeakerItem):
    memberID: int
    speechToken: str

    @classmethod
    def from_speaker(cls, item: SpeakerItem, member_id: int, speech_token: str) -> "SpeakerDbItem":
        return cls(**item.model_dump(), memberID=member_id, speechToken=speech_token)

class SpeakerDbScoreItem(SpeakerDbItem):
    score: float

class IssueItem(BaseModel):
    issueID: str
    nameOfHouse: Optional[str] = None
    nameOfMeeting: Optional[str] = None
    issue: Optional[str] = None
    date: Optional[str] = None

class SpeakerIssueSummaryItem(IssueItem):
    memberID: int
    summary: str


class KokkaiDbRepository:
    def __init__(self):
        self.db_path = "./kokkai.duckdb"
    
    def setup(self, is_update_fts: bool = True):
        with self.connect() as conn:
            conn.execute("INSTALL fts;")
            conn.execute("LOAD fts;")
            # 議員テーブル
            conn.execute("""
            CREATE SEQUENCE IF NOT EXISTS member_id_seq START 1;

            CREATE TABLE IF NOT EXISTS members (
                id BIGINT PRIMARY KEY DEFAULT nextval('member_id_seq'),
                name VARCHAR NOT NULL,
                kana VARCHAR NOT NULL,
                kaiha VARCHAR
            );
            """)
            # 発話者テーブル
            conn.execute("""
            CREATE TABLE IF NOT EXISTS speakers (
                speechID VARCHAR NOT NULL PRIMARY KEY,
                memberID BIGINT,
                issueID VARCHAR NOT NULL,
                imageKind VARCHAR,
                searchObject INTEGER,
                session INTEGER,
                nameOfHouse VARCHAR,
                nameOfMeeting VARCHAR,
                issue VARCHAR,
                date VARCHAR,
                speechOrder INTEGER,
                speakerGroup VARCHAR,
                speakerPosition VARCHAR,
                speakerRole VARCHAR,
                speech TEXT,
                speechToken TEXT,
                FOREIGN KEY (memberID) REFERENCES members(id)
            );
            """)
            # 会議ごとの発言者のsummary
            conn.execute("""
            CREATE TABLE IF NOT EXISTS speaker_summaries (
                memberID BIGINT,
                issueID VARCHAR NOT NULL,
                nameOfHouse VARCHAR,
                nameOfMeeting VARCHAR,
                issue VARCHAR,
                date VARCHAR,
                summary TEXT,
                FOREIGN KEY (memberID) REFERENCES members(id)
            );
            """)

            # インデックスの再作成
            if is_update_fts:
                self.update_fts_index(conn)


    def update_fts_index(self, conn):
        try:
            conn.execute("""
            PRAGMA drop_fts_index('speakers');
            """)
        except duckdb.Error as e:
            print(e)
            pass
        conn.execute("""
        PRAGMA create_fts_index(
            'speakers',
            'speechID',
            'speechToken'
        );
        """)

    def connect(self):
        con = duckdb.connect(self.db_path)
        return con

    def get_member_id(self, conn, name: str)->Optional[int]:
        res = conn.execute("""
        SELECT id FROM members WHERE name = ?;
        """, (name,)).fetchone()
        return res[0] if res else None

    def append_member(self, conn, name: str, kana: str, kaiha: Optional[str]=None)->int:
        res = conn.execute("""
        INSERT INTO members (name, kana, kaiha)
        VALUES (?, ?, ?)
        RETURNING id;
        """, (name, kana, kaiha)).fetchone()
        return res[0]

    def append_speaker(self, conn, items: list[SpeakerDbItem]):
        rows = [(
            item.speechID,
            item.memberID,
            item.issueID,
            item.imageKind,
            item.searchObject,
            item.session,
            item.nameOfHouse,
            item.nameOfMeeting,
            item.issue,
            item.date,
            item.speechOrder,
            item.speakerGroup,
            item.speakerPosition,
            item.speakerRole,
            item.speech,
            item.speechToken
        ) for item in items]
        conn.executemany("""
        INSERT INTO speakers (
            speechID,
            memberID,
            issueID,
            imageKind,
            searchObject,
            session,
            nameOfHouse,
            nameOfMeeting,
            issue,
            date,
            speechOrder,
            speakerGroup,
            speakerPosition,
            speakerRole,
            speech,
            speechToken
        ) VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? );
        """, rows)

    def get_issue_speeches(self, conn, speaker: str, issue_id: str, limit: int)->Iterator[SpeakerDbItem]:
        member_id = self.get_member_id(conn, speaker)
        if member_id is None:
            return
        rows = conn.execute(
        """
        SELECT
            speechID,
            memberID,
            issueID,
            imageKind,
            searchObject,
            session,
            nameOfHouse,
            nameOfMeeting,
            issue,
            date,
            speechOrder,
            speakerGroup,
            speakerPosition,
            speakerRole,
            speech,
            speechToken,
        FROM speakers
        WHERE memberID = ? AND issueID = ?
        ORDER BY speechID ASC
        LIMIT ?;
        """,
        (member_id, issue_id, limit),
        ).fetchall()
        for row in rows:
            item = SpeakerDbItem(
                speechID=row[0],
                memberID=row[1],
                issueID=row[2],
                imageKind=row[3],
                searchObject=row[4],
                session=row[5],
                nameOfHouse=row[6],
                nameOfMeeting=row[7],
                issue=row[8],
                date=row[9],
                speechOrder=row[10],
                speakerGroup=row[11],
                speakerPosition=row[12],
                speakerRole=row[13],
                speech=row[14],
                speechToken=row[15],
            )
            yield item

    def get_speaker_issues(self, conn, speaker: str)->list[IssueItem]:
        member_id = self.get_member_id(conn, speaker)
        if member_id is None:
            return []
        rows = conn.execute(
        """
        SELECT DISTINCT issueID, issue, nameOfHouse, nameOfMeeting, date FROM speakers
        WHERE memberID = ?;
        """,
        (member_id,),
        ).fetchall()
        return [
            IssueItem(
                issueID=row[0],
                issue=row[1],
                nameOfHouse=row[2],
                nameOfMeeting=row[3],
                date=row[4],
            )
            for row in rows
        ]
    
    def get_speaker_issue_summary(self, conn, member_id: id, issueID: str)->Optional[SpeakerIssueSummaryItem]:
        row = conn.execute(
        """
        SELECT
            memberID,
            issueID,
            issue,
            nameOfHouse,
            nameOfMeeting,
            date,
            summary
        FROM speaker_summaries
        WHERE memberID = ? AND issueID = ?;
        """,
        (member_id,issueID, ),
        ).fetchone()
        if not row:
            return None
        return SpeakerIssueSummaryItem(
            memberID = row[0],
            issueID = row[1],
            issue = row[2],
            nameOfHouse = row[3],
            nameOfMeeting = row[4],
            date = row[5],
            summary = row[6]
        )
    def append_speaker_issue_summary(self, conn, item: SpeakerIssueSummaryItem):
        conn.execute("""
        INSERT INTO speaker_summaries (
            memberID,
            issueID,
            issue,
            nameOfHouse,
            nameOfMeeting,
            date,
            summary
        ) VALUES ( ?, ?, ?, ?, ?, ?, ? );
        """, (
            item.memberID,
            item.issueID,
            item.issue,
            item.nameOfHouse,
            item.nameOfMeeting,
            item.date,
            item.summary,
        ))

    def search_speeches(self, conn, speaker: str, query: str, limit: int)->Iterator[SpeakerDbScoreItem]:
        member_id = self.get_member_id(conn, speaker)
        if member_id is None:
            return
        rows = conn.execute(
        """
        SELECT
            speechID,
            memberID,
            issueID,
            imageKind,
            searchObject,
            session,
            nameOfHouse,
            nameOfMeeting,
            issue,
            date,
            speechOrder,
            speakerGroup,
            speakerPosition,
            speakerRole,
            speech,
            speechToken,
            score
        FROM (
            SELECT
                speechID,
                memberID,
                issueID,
                imageKind,
                searchObject,
                session,
                nameOfHouse,
                nameOfMeeting,
                issue,
                date,
                speechOrder,
                speakerGroup,
                speakerPosition,
                speakerRole,
                speech,
                speechToken,
                fts_main_speakers.match_bm25(speechID, ?) AS score
            FROM speakers
            WHERE memberID = ?
        ) t
        WHERE score IS NOT NULL
        ORDER BY score DESC
        LIMIT ?;
        """,
        (query, member_id, limit),
        ).fetchall()
        for row in rows:
            item = SpeakerDbScoreItem(
                speechID=row[0],
                memberID=row[1],
                issueID=row[2],
                imageKind=row[3],
                searchObject=row[4],
                session=row[5],
                nameOfHouse=row[6],
                nameOfMeeting=row[7],
                issue=row[8],
                date=row[9],
                speechOrder=row[10],
                speakerGroup=row[11],
                speakerPosition=row[12],
                speakerRole=row[13],
                speech=row[14],
                speechToken=row[15],
                score=row[16],
            )
            yield item


In [132]:
db_repository = KokkaiDbRepository()
db_repository.setup(False)

In [ ]:
from sudachipy import tokenizer
from sudachipy import dictionary

class KokkaiSpeakerService:
    def __init__(self, db_repository: KokkaiDbRepository, kokkai_api: KokkaiSpeacherApi, open_ai_service: KokkaiOpenAIService):
        self.db_repository = db_repository
        self.kokkai_api = kokkai_api
        self.open_ai_service = open_ai_service
        self.tokenizer = dictionary.Dictionary().create()


    def import_speakers(self, speaker_name: str, speaker_kana: str, speaker_kaiha: str, from_date: Optional[str]=None, until_date: Optional[str]=None):
        with self.db_repository.connect() as conn:
            member_id = self.db_repository.get_member_id(conn, speaker_name)
            if member_id is not None:
                print(f"{speaker_name} は登録済みである")
                return 
            conn.execute("BEGIN;")
            member_id = self.db_repository.append_member(conn, speaker_name, speaker_kana, speaker_kaiha)
            items = []
            for item in self.kokkai_api.get_speak(speaker_name, from_date, until_date):
                tokens = " ".join([m.surface() for m in self.tokenizer.tokenize(item.speech, tokenizer.Tokenizer.SplitMode.C)])
                db_item = SpeakerDbItem.from_speaker(item, member_id, tokens)
                items.append(db_item)
                if len(items) >= 100:
                    self.db_repository.append_speaker(conn, items)
                    conn.commit()
                    conn.execute("BEGIN;")
                    items = []
            if items:
                self.db_repository.append_speaker(conn, items)
            conn.commit()
            self.db_repository.update_fts_index(conn)

    def make_speaker_issue_summary(self, speaker_name: str)->list[SpeakerIssueSummaryItem]:
        result = []
        with self.db_repository.connect() as conn:
            member_id = self.db_repository.get_member_id(conn, speaker_name)
            if member_id is None:
                print(f"{speaker_name} は未登録である")
                return
            for issue in db_repository.get_speaker_issues(conn, speaker_name):
                issue_summary = self.db_repository.get_speaker_issue_summary(conn, member_id, issue.issueID)
                if issue_summary:
                    print(f"{speaker_name}の{issue.issueID}のサマリは作成済み")
                    result.append(issue_summary)
                    continue
                speeches = []
                for speech in db_repository.get_issue_speeches(conn, speaker_name, issue.issueID, 100):
                    speeches.append(speech.speech)
                summary =self.open_ai_service.get_issue_summary(speeches)
                issue_summary = SpeakerIssueSummaryItem(
                    **issue.model_dump(),
                    memberID=member_id,
                    summary=summary
                )
                self.db_repository.append_speaker_issue_summary(conn, issue_summary)
                # まとめてコミットすると、異常終了した際のopenAIの使用量がもったいないのでcommitしておく
                conn.commit()
                result.append(issue_summary)
        return result

    def make_speaker_text(self, speaker_name: str, output_path: str):
        items = service.make_speaker_issue_summary(speaker_name)
        messages = [item.summary for item in items]
        summary = self.open_ai_service.get_speaker_summary(messages)
        with open(output_path, "w") as f:
            f.write("## 質疑一覧\n")
            for item in items:
                f.write(f" - {item.date}  {item.nameOfHouse} {item.nameOfMeeting} {item.issue} {item.summary}\n")
            f.write("## まとめ\n")
            f.write(f"{summary}\n")


In [12]:
from_date = "2022-01-01"
until_date = "2026-01-31"
service = KokkaiSpeakerService(db_repository, KokkaiSpeacherApi(), KokkaiOpenAIService())
service.import_speakers("玉木雄一郎", "たまきゆういちろう", "国民", from_date, until_date)


member_id: None
Calling API with params: {'speaker': '玉木雄一郎', 'maximumRecords': 100, 'recordPacking': 'json', 'startRecord': 1, 'from': '2022-01-01', 'until': '2026-01-31'}
Received 100 records 275
{'speechID': '121905261X00720251210_216', 'issueID': '121905261X00720251210', 'imageKind': '会議録', 'searchObject': 216, 'session': 219, 'nameOfHouse': '衆議院', 'nameOfMeeting': '予算委員会', 'issue': '第7号', 'date': '2025-12-10', 'closing': None, 'speechOrder': 216, 'speaker': '玉木雄一郎', 'speakerYomi': 'たまきゆういちろう', 'speakerGroup': '国民民主党・無所属クラブ', 'speakerPosition': None, 'speakerRole': None, 'speech': '○玉木委員\u3000国民民主党の玉木雄一郎です。\r\n\u3000総理、よろしくお願いいたします。\r\n\u3000今日は十二月十日、あしたが十二月十一日です、当たり前ですけれども。ただ、一年前の十二月十一日は大切な日でありまして、我が党と、そして御党自民党、また公明党、当時与党だった公明党さん、三党の幹事長間でいわゆる三党合意を結んだのが、ちょうど約一年前の十二月十一日でありました。二つのことを決めました。一つは、ガソリンの暫定税率を廃止するということ、二つ目が、いわゆる百三万の年収の壁を百七十八万円を目指して引き上げる、当時、来年中に引き上げるということ、二つを合意いたしました。\r\n\u3000このうちの一つのガソリンの暫定税率の廃止は、最終最後、総理やまた片山大臣にもリーダーシップを発揮していただいて、また、与野党のそれぞれの関係の先生方にも本当に御尽力をいただいて、年内廃止が決まりまし

In [22]:
from sudachipy import tokenizer
from sudachipy import dictionary


tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C
tokens = [m.surface() for m in tokenizer_obj.tokenize("社会保障", mode)]
print(tokens)

['社会保障']


In [20]:
for row in db_repository.search_speeches(
    db_repository.connect(),
    speaker="玉木雄一郎",
    query="社会保障",
    limit=10
):
    print(row)


speaker=None speakerYomi=None speechID='121724293X00320250611_030' issueID='121724293X00320250611' imageKind='会議録' searchObject=30 session=217 nameOfHouse='両院' nameOfMeeting='国家基本政策委員会合同審査会' issue='第3号' date='2025-06-11' speechOrder=30 speakerGroup='国民民主党・無所属クラブ' speakerPosition=None speakerRole=None speech='○玉木雄一郎君\u3000ちょっと発言がぶれていますね。\r\n\u3000もう一回聞きます。税収の上振れを還元することはできない、できる状態にない。あるなら、多分、総理の考えだと、国債の償還に回すか、あるいは、そういったお金については、ほかのものにしっかり、まあ、社会保障に充てていくと。赤字国債を発行していますからね。ということだと思うんですが、もう一回伺います。\r\n\u3000政府としては、今後も、税収の上振れ分を還元する形で現金給付することはないと明言できますか。' startPage=0 speechURL=None meetingURL=None pdfURL=None memberID=1 speechToken='○ 玉木 雄一郎 君 \u3000 ちょっと 発言 が ぶれ て い ます ね 。 \r\n\u3000 もう 一回 聞き ます 。 税収 の 上振れ を 還元 する こと は でき ない 、 できる 状態 に ない 。 ある なら 、 多分 、 総理 の 考え だ と 、 国債 の 償還 に 回す か 、 あるいは 、 そう いっ た お金 に つい て は 、 ほか の もの に しっかり 、 まあ 、 社会保障 に 充て て いく と 。 赤字 国債 を 発行 し て い ます から ね 。 と いう こと だ と 思う ん です が 、 もう 一回 伺い ます 。 \r\n\u3000 政府 と し て は 、 今後 も 、 税収 の 上振れ 分 を 還元 する 形 で 現金給付 する こと は ない と 明言 でき ます

In [8]:
import csv
import re
from tqdm import tqdm

from_date = "2022-01-01"
until_date = "2026-01-31"
service = KokkaiSpeakerService(db_repository, KokkaiSpeacherApi(), OpenAiService())


with open("2025_members.csv") as f:
    reader = csv.reader(f)
    next(reader)  # ヘッダーをスキップ
    for row in tqdm(reader):
        name = re.sub(r"\s+", "", row[0])[:-1]
        kana = re.sub(r"\s+", "", row[1])
        kaiha = re.sub(r"\s+", "", row[2])
        service.import_speakers(name, kana, kaiha, from_date, until_date)


465it [5:28:08, 42.34s/it]


In [14]:
with open("2024_members.csv") as f:
    reader = csv.reader(f)
    next(reader)  # ヘッダーをスキップ
    for row in tqdm(reader):
        name = re.sub(r"\s+", "", row[0])[:-1]
        kana = re.sub(r"\s+", "", row[1])
        kaiha = re.sub(r"\s+", "", row[2])
        service.import_speakers(name, kana, kaiha, from_date, until_date)

4it [00:00, 18.35it/s]

逢沢一郎 は登録済みである
青柳仁士 は登録済みである
青柳陽一郎 は登録済みである
青山周平 は登録済みである
青山大人 は登録済みである
赤木正幸 は登録済みである


10it [00:00, 20.85it/s]

赤澤亮正 は登録済みである
赤羽一嘉 は登録済みである
あかま二郎 は登録済みである
赤嶺政賢 は登録済みである
秋葉賢也 は登録済みである


17it [00:00, 25.76it/s]

...

早稲田ゆき は登録済みである
和田有一朗 は登録済みである


459it [1:04:58, 56.87s/it]

渡辺周 は登録済みである
渡辺創 は登録済みである


463it [1:06:43,  8.65s/it]

鰐淵洋子 は登録済みである


In [16]:
member2025 = set()
member2024 = set()

with open("2024_members.csv") as f:
    reader = csv.reader(f)
    next(reader)  # ヘッダーをスキップ
    for row in tqdm(reader):
        member2024.add(re.sub(r"\s+", "", row[0])[:-1])
with open("2025_members.csv") as f:
    reader = csv.reader(f)
    next(reader)  # ヘッダーをスキップ
    for row in tqdm(reader):
        member2025.add(re.sub(r"\s+", "", row[0])[:-1])

diff_members = member2024 - member2025
print("2024->2025年に削除された議員:", len(diff_members))


463it [00:00, 149072.14it/s]
465it [00:00, 77266.12it/s]

2024->2025年に削除された議員: 123


In [38]:
for row in db_repository.search_speeches(
    db_repository.connect(),
    speaker="大空幸星",
    query="児童 福祉",
    limit=1000
):
    print(row.speechID, row.date, row.score, row.speakerPosition,row.speakerRole, row.speech)


121315363X00220240214_039 2024-02-14 3.4286569037270143 ＮＰＯ法人あなたのいばしょ理事長 参考人 ○参考人（大空幸星君）　ありがとうございます。
　自治会とか町会を既に廃止しているところもあるにもかかわらず、民生委員になるのに今度自治会の推薦が必要みたいな、ないのに推薦が必要みたいになっているところもあったりして、じゃ、どうやるんだみたいなところがやっぱりあるという現状ありますよね。
　やっぱり欠員が、僕の地元の江東区も、やっぱり欠員が出ているにもかかわらず、やっぱり推薦が、湾岸部なんかは推薦が必要ですと。いや、でも湾岸部に自治会なんかないわけですね、高層マンションばっかりのところに。やっぱりそういう現状が今、いまだにあるということだと思います。
　本来は、この今の民生委員さん、児童委員さん、主任児童委員さんの枠組みの中に若い人が入ればいいだけの話なんです。ただ、とはいえ、百年たって、例えば様々な認証業務がありますね、民生委員さんは。具体的な対面の支援だけじゃなくて、やっぱり公的な仕組みですし、当然これは厚生労働大臣の任命で、天皇陛下から勲章もいただけるというような、そういう仕組みになっていますから、なかなかやっぱりこの既存の民生委員さん、児童委員さん、主任児童委員さんからしても、今のその仕事を若い人がやるというのは無理だろうというような感覚は分からなくもないと。
　となったときに、必要なのは、やはり今、子供たち、若者たちが抱えている悩みに寄り添う存在です。この存在として本来例えば児童委員さんなんかができてきたはずが、じゃ、児童委員さん、主任児童委員さん、民生委員さんと同じ人がやっているというケースがほとんどだろうと思うんですね。これは、でも、制度としてはまた別の制度だったりするわけで。やっぱりそうしたことを考えると、子供たち、若者たちに関する悩み、その地域に住んでいる者、それに関しては、少し年上の世代の例えば二十代が三年から四年の任期でやると。
　これは、やはり最初は対面じゃなくてオンラインでやったらいいと思うんです。例えば、民生委員さんがやっている仕事の中には、その認証業務以外にも、相談業務であるとか援助業務であるとか情報提供であるとか状況把握みたいなことがあるわけですよね。地方だったら、どこどこのおばあち

In [75]:
from statistics import median

speaker = "大空幸星"
with db_repository.connect() as conn:
    for item in db_repository.get_speaker_issues(conn, speaker):
        speeches = []
        text_size_list = []
        for speech in db_repository.get_issue_speeches(conn, speaker, item.issueID, 1000):
            speeches.append(speech.speech)
            text_size_list.append(len(speech.speech))
        print(item, len(speeches), median(text_size_list), max(text_size_list), sum(text_size_list))


issueID='121904601X00320251127' nameOfHouse='衆議院' nameOfMeeting='総務委員会' issue='第3号' date='2025-11-27' 8 668.0 1237 6255
issueID='121315363X00220240214' nameOfHouse='参議院' nameOfMeeting='国民生活・経済及び地方に関する調査会' issue='第2号' date='2024-02-14' 9 1371 10049 21048
issueID='121705007X00720250408' nameOfHouse='衆議院' nameOfMeeting='農林水産委員会' issue='第7号' date='2025-04-08' 5 739 1447 4164
issueID='121703968X00320250326' nameOfHouse='衆議院' nameOfMeeting='外務委員会' issue='第3号' date='2025-03-26' 6 1067.0 1621 5623
issueID='121904319X00320251126' nameOfHouse='衆議院' nameOfMeeting='国土交通委員会' issue='第3号' date='2025-11-26' 6 636.0 829 3502
issueID='121703968X01320250528' nameOfHouse='衆議院' nameOfMeeting='外務委員会' issue='第13号' date='2025-05-28' 5 706 1399 4081
issueID='121705270X00220250228' nameOfHouse='衆議院' nameOfMeeting='予算委員会第四分科会' issue='第2号' date='2025-02-28' 13 489 1408 7736
issueID='121104889X01620230426' nameOfHouse='衆議院' nameOfMeeting='内閣委員会' issue='第16号' date='2023-04-26' 16 977.0 5795 21148
issueID='121705266

In [72]:
open_ai_service = KokkaiOpenAIService()
with db_repository.connect() as conn:
    for item in db_repository.get_speaker_issues(conn, speaker):
        speeches = []
        for speech in db_repository.get_issue_speeches(conn, speaker, item.issueID, 100):
            speeches.append(speech.speech)
            print(speech)
        print('=====================')
        res = open_ai_service.get_issue_summary(speeches)
        print(item, res)
        break

speaker=None speakerYomi=None speechID='121904601X00320251127_012' issueID='121904601X00320251127' imageKind='会議録' searchObject=12 session=219 nameOfHouse='衆議院' nameOfMeeting='総務委員会' issue='第3号' date='2025-11-27' speechOrder=12 speakerGroup='自由民主党・無所属の会' speakerPosition=None speakerRole=None speech='○大空委員\u3000おはようございます。自由民主党の大空幸星でございます。\r\n\u3000今日は、令和二、三、四、五と、四年分まとめてのＮＨＫ決算ということで、皆さんのお手元にも大変多くの資料が配られていると思いますが、単年度決算を連続して比較するというのはなかなか難しいという前提に立った上で、国民目線のためのＮＨＫとしてあり続けていただきたい、その思いで質問をさせていただきます。\r\n\u3000まず、ＮＨＫの予算そして業務報告書には、総務大臣意見が付されるということになっております。令和二、三、四、五と、業務報告書に対する総務大臣意見において繰り返し言及をされている言葉がございます。それが、より精緻な収支予算の編成に努めることが望まれると。これは、全ての業務報告書で全く同じような文言が総務大臣から付されているわけでございます。令和二から四年度は、赤字予算を掲げながら実績では黒字となった。令和五年度は逆に赤字となっています。総務省としては、収支予算の精度向上を求めておりますけれども、毎年同じような指摘になってしまっている。\r\n\u3000まずは、このことについて、すなわち、収支予算の精度向上の改善がなかなか見えづらいんじゃないか、こういう声についてどういうふうに受け止めておられるか、お聞かせください。' startPage=0 speechURL=None meetingURL=None pdfURL=None memberID=84 speechToken='○ 大空 委員 \u3000 おはようございます 。 自由民主党 の 大空 幸 星 で ござい ます 

In [133]:
service = KokkaiSpeakerService(db_repository, KokkaiSpeacherApi(), KokkaiOpenAIService())
speaker = "大空幸星"
items = service.make_speaker_issue_summary(speaker)
for item in items:
    print(item)

大空幸星の121104889X01620230426のサマリは作成済み
大空幸星の121705007X00720250408のサマリは作成済み
大空幸星の121705266X00120250227のサマリは作成済み
大空幸星の121904601X00320251127のサマリは作成済み
大空幸星の121315363X00220240214のサマリは作成済み
大空幸星の121703968X00320250326のサマリは作成済み
大空幸星の121904319X00320251126のサマリは作成済み
大空幸星の121703968X01320250528のサマリは作成済み
大空幸星の121705270X00220250228のサマリは作成済み
issueID='121104889X01620230426' nameOfHouse='衆議院' nameOfMeeting='内閣委員会' issue='第16号' date='2023-04-26' memberID=84 summary='孤独・孤立対策を国家の中核として予防重視・実務運用を織り込んだ法制・制度設計を問う質疑です。具体的には、24時間 anonチャット等の広域窓口と地域支援の連携強化、自治体の協議会と推進本部の機能分担、複数年の安定財源確保とNPOの持続可能性、AI等を活用したリスク判定と待機対応の適正化、学校・民生委員・企業等との協働の促進、スティグマ対策の教育・広報・制度設計、オンラインとオフラインの接続強化、在外邦人の支援・アクセス改善、現場の実態把握に基づく運用ガイドライン整備、自治体間連携の枠組みづくりなど、実務的な連携強化と制度的整備の必要性を問う趣旨です。'
issueID='121705007X00720250408' nameOfHouse='衆議院' nameOfMeeting='農林水産委員会' issue='第7号' date='2025-04-08' memberID=84 summary='木材の需要喚起について、環境性だけでなく木の効用「木力」＝生活・健康・湿度・リラックス効果などを訴え、住宅以外の中高層木造建築や構造材利用の拡大、企業の活用促進インセンティブ設計など民間の実践を後押しする政策をどう位置づけるべきかを問う。あわせて政府備蓄米の子供食堂等への無償交付について、申請

In [104]:
open_ai_service = KokkaiOpenAIService()
messages = [item.summary for item in items]
res = open_ai_service.get_speaker_summary(messages)
print(res)

孤独・孤立対策を国家の柱とし、予防重視の法制度と地域づくりを推進。24時間窓口・AIリスク判定・オンライン居場所を連携させ、学校・民生委員・企業と協働して支援を迅速化。スティグマ克服と啓発、現場浸透の居場所づくりと多年度財源・透明な評価を確保。木材の需要喚起は“木力”として健康・生活を訴え、中高層木造・構造材拡大と企業インセン設計を促進。備蓄米の無償交付は公平性・データ活用で拡大。国際協力は信頼回復と成果連動投融資の適正運用、在外邦人支援を強化。


In [136]:
open_ai_service = KokkaiOpenAIService()
service = KokkaiSpeakerService(db_repository, KokkaiSpeacherApi(), open_ai_service)
speaker = "西村智奈美"
items = service.make_speaker_text(speaker, output_path=f"./outputs/{speaker}.txt")


西村智奈美の121705206X01920250604のサマリは作成済み
西村智奈美の121704320X00120250514のサマリは作成済み
西村智奈美の121705206X00120250307のサマリは作成済み
西村智奈美の121305254X03320240606のサマリは作成済み
西村智奈美の121304260X01420240419のサマリは作成済み
西村智奈美の121305253X00320240408のサマリは作成済み
西村智奈美の121205261X00220231027のサマリは作成済み
西村智奈美の121004260X00720221109のサマリは作成済み
西村智奈美の120804376X02020220608のサマリは作成済み
西村智奈美の121705206X02320250613のサマリは作成済み
西村智奈美の121705206X02020250606のサマリは作成済み
西村智奈美の121705254X02420250508のサマリは作成済み
西村智奈美の121705206X00220250312のサマリは作成済み
西村智奈美の121205261X00820231208のサマリは作成済み
西村智奈美の121205206X00620231205のサマリは作成済み
西村智奈美の121205206X00420231124のサマリは作成済み
西村智奈美の121104260X00820230412のサマリは作成済み
西村智奈美の121104260X00420230322のサマリは作成済み
西村智奈美の121105261X00920230209のサマリは作成済み
西村智奈美の121705206X01720250528のサマリは作成済み
西村智奈美の121605206X00120241206のサマリは作成済み
西村智奈美の121304260X00620240327のサマリは作成済み
西村智奈美の121105367X01720230705のサマリは作成済み
西村智奈美の121104536X00320230330のサマリは作成済み
西村智奈美の121005261X00720221128のサマリは作成済み
西村智奈美の121705206X02420250617のサマリは作成済み
西村智奈美の121705254X02820250522のサマリは作成済み
西

In [ ]:
speaker = "西村智奈美"
with db_repository.connect() as conn:
    member_id = db_repository.get_member_id(conn, speaker)
    for issue in db_repository.get_speaker_issues(conn, speaker):
        issue_summary = db_repository.get_speaker_issue_summary(conn, member_id, issue.issueID)
        print(issue_summary)
        #if not issue_summary:
        #    continue
        #row = conn.execute(
        #"""
        #SELECT
        #    date
        #FROM speakers
        #WHERE issueID = ?;
        #""",
        #(issue_summary.issueID, ),
        #).fetchone()
        #print("    ", row)
        #conn.execute(
        #"""
        #UPDATE speaker_summaries
        #    SET date = ?
        #WHERE memberID=? AND issueID = ?;
        #""",
        #(row[0], member_id, issue_summary.issueID),
        #)


issueID='121705206X02520250620' nameOfHouse='衆議院' nameOfMeeting='法務委員会' issue='第25号' date='2025-06-20' memberID=312 summary='西村委員長は、本会期付託請願370件を一括議題とし、審査方法は簡略化して直ちに採決することを提案・了承した。裁判所の人的・物的充実に関する39件は全件採択・内閣送付と決定。委員会報告書作成は委員長に一任。陳情31件・意見書401件の趣旨は配布資料で周知。閉会中審査の件では、複数の民法改正案・刑事訴訟法改正案等を議長へ申出することに賛成多数で可決。閉会中の参考人出席・委員派遣は委員長に一任。家族の氏に関する議論は速やかに継続審議し、今秋臨時国会で審議方針と合意。散会。'
issueID='121705206X01520250516' nameOfHouse='衆議院' nameOfMeeting='法務委員会' issue='第15号' date='2025-05-16' memberID=312 summary='本日、裁判所の司法行政・法務行政・検察行政、国内治安、人権擁護に関する調査を進めるため、警察庁刑事局組織犯罪対策部長ら計12名および最高裁判所事務総局刑事局長ら計1名の出席説明を求め、聴取することを決定した。質疑は順次行われ、政府提出の譲渡担保契約及び所有権留保契約に関する法律案と、それに伴う関係法の整備等の趣旨説明を鈴木法務大臣から聴取する。今後の日程も設定されている。'
issueID='121705206X00320250314' nameOfHouse='衆議院' nameOfMeeting='法務委員会' issue='第3号' date='2025-03-14' memberID=312 summary='本日の質疑は、内閣提出の裁判所職員定員法の一部改正案について、政府出席者の説明を聴取し、答弁の訂正・修正を指摘して内容の正確さと要点の把握、時間配分の適正さを検証する趣旨で行われた。警察庁・法務省などの担当者出席を承認し、答弁の大声化や理事会協議事項への対応にも言及。質疑後、附帯決議を付す動議を採択し、原案通り可決。報告書作成を委員長に一任。次回は18日。'
issueID='121605254X